# Simulations of scattering event according to original dynamics

In [4]:
using NBInclude
using Ranges

include("./functions.jl")
using TwoBodyScattering

ArgumentError: ArgumentError: Package TwoBodyScattering not found in current path, maybe you meant `import/using .TwoBodyScattering`.
- Otherwise, run `import Pkg; Pkg.add("TwoBodyScattering")` to install the TwoBodyScattering package.

In [2]:
#Define parameters for simulations

global v0::Float64 = 10.0; # Self-propulsion speed
global int_range::Float64 = 1; #Range of interactions
global gamma::Float64 = 1.0; # Intensity of interactions
global dtf::Float64 = 1e-4;

In [ ]:
#Define required vectors and variables for saving results

total_N::Int64 = 100;
dphi::Float64 = 2*pi/total_N;
phi1seq = range(-pi, pi, total_N);
phi2seq = range(-pi, pi, total_N);
deltaps = Float64[];
dinit = Float64[];

# Simulate the scattering events and save outgoing angle

generate!(dinit, deltaps,
        dphi,
        gamma, v0, int_range, 0.0,
        dtf);

In [ ]:
pos_deltaps = Float64[];
pos_dinit = Float64[];
for i in 1:length(deltaps)
   if deltaps[i] < 0
        push!(pos_dinit, dinit[i]);
        push!(pos_deltaps, deltaps[i]);
    end
end
    

In [ ]:
scatter(dinit,deltaps)

In [ ]:
#Run simulations at fixed v_0 for different values of dissipation
using Plots

#Variables for simulation
total_N::Int64 = 100;
dphi::Float64 = 2*pi/total_N;
phi1seq::Vector{Float64} = range(-pi, pi, total_N);
phi2seq::Vector{Float64} = range(-pi, pi, total_N);
deltaps::Vector{Float64} = Float64[];
thetatot::Vector{Float64} = Float64[];
dinit::Vector{Float64} = Float64[];
v0::Float64 = 100;

#Ranges of values for dissipation intensity
cranges = 0.1:0.1:1;

#Colours palette
palettes = range(colorant"red", stop=colorant"green", length=length(cranges));

#Generate first plot

generate!(dinit, thetatot, deltaps,
        dphi,
        gamma, v0, int_range, 0.0,
        dtf);
rangex = -(pi):dphi:(pi-dphi);
avgpdp = zeros(length(rangex));
totn = zeros(length(rangex));
for x in 2:length(rangex)
    for i in 1:length(dinit)
        if modulate(dinit[i]) <= rangex[x] && modulate(dinit[i]) > rangex[x-1]
            avgpdp[x]+=deltaps[i];
            totn[x]+=1;
        end
    end
end
avgpdp = avgpdp./totn;

p = plot(rangex[Int(total_N/2):(total_N-1)], avgpdp[Int(total_N/2):(total_N-1)], xticks=pitick(-pi,pi,4), color = palettes[1])
xlabel!(p, "Delta")
ylabel!(p, "pdp")

#Generate plots for all ranges of intensities

for i in 1:length(cranges)
   generate!(dinit, thetatot, deltaps,
        dphi,
        gamma, v0, int_range, cranges[i],
        dtf);
    
    avgpdp = zeros(length(rangex));
    totn = zeros(length(rangex));
    for x in 2:length(rangex)
        for i in 1:length(dinit)
            if modulate(dinit[i]) <= rangex[x] && modulate(dinit[i]) > rangex[x-1]
                avgpdp[x]+=deltaps[i];
                totn[x]+=1;
            end
        end
    end
    avgpdp = avgpdp./totn;

    plot!(p, rangex[Int(total_N/2):(total_N-1)], avgpdp[Int(total_N/2):(total_N-1)], xticks=pitick(-pi,pi,4), color = palettes[i])
end

In [ ]:
p
#savefig("pdpv0100VaringDissfrom0to1.pdf")

In [ ]:
phi10 = pi/2+0.1;
phi20 = -1.0;
pdp, phi1t, phi2t, a11, a22 = scatteringmovie(phi10,phi20,1.0,1e-7, 
                            1.0, 100.0, 0.0);

In [ ]:
using Plots

lims = 10;
steps = 1000;
animd = @animate for i in 1:steps:length(a11)
    plot(Tuple.(a11[i]), xlims=(-lims,lims), ylims=(-5,5))
    plot!(Tuple.(a22[i]))
end

In [ ]:
gif(animd, fps=30)

In [ ]:
print(phi1t, " ", phi10,"\n")
print(phi2t, " ", phi20,"\n")

In [ ]:
print(phi1t-phi2t, "\n")
print(phi10-phi20, "\n")
print(phi10-phi20-2*pi)